In [1]:
%%time
from google.cloud import bigquery

bqclient= bigquery.Client()

query= """
  SELECT *
  FROM eurekathon-hub.CEIIA.shared_micromobility
"""

df0= (
  bqclient.query(query)
          .result()
          .to_dataframe()
)

df0.head()

CPU times: user 1.2 s, sys: 632 ms, total: 1.83 s
Wall time: 4.49 s


,vehicle_id,session_id,type,start_time,end_time,distance,saved_emissions,route
0,256,5a6b9477-1457-4813-88cb-1a0bf88e2282,E-SCOOTER,1559070584000,1559070951000,1472.000000000,135.041280000,NULL
1,256,2a57ae36-e551-4073-b8f4-9ef7e1ede81f,E-SCOOTER,1560191671000,1560191872000,80.000000000,7.367200000,01040000004A0000000101000000000000A0BD6E21C000...
2,256,24b6a823-03eb-4270-b255-8e7940e6bd2b,E-SCOOTER,1559738931000,1559739528000,2441.000000000,224.791690000,0104000000490000000101000000000000800A6121C000...
3,256,0b85ebb4-f6b3-45b3-9f99-b89aaede71a0,E-SCOOTER,1559519594000,1559520691000,3867.000000000,354.758580000,0104000000730000000101000000000000A00B6121C000...
4,256,a6722d8d-7c8a-42db-9590-96ccf92cb34e,E-SCOOTER,1560097302000,1560097450000,91.000000000,8.380190000,01040000002A0000000101000000000000A0D27021C000...


In [2]:
df0.shape

(158444, 8)

# EDA

In [3]:
import pandas as pd
import numpy as np

## eurekathon-hub.CEIIA.shared_micromobility


In [4]:
df= df0.copy()

In [5]:
pd.Series(df.columns)

0         vehicle_id
1         session_id
2               type
3         start_time
4           end_time
5           distance
6    saved_emissions
7              route
dtype: object

In [6]:
df.dtypes

vehicle_id          int64
session_id         object
type               object
start_time         object
end_time           object
distance           object
saved_emissions    object
route              object
dtype: object

FEATURE | DESCRIPTION | TYPE
---:|--- |:---:
vehicle_id: | identificação do veículo | integer 64
session_id: | identificação da sessão | string
type: | tipo de veículo | string
start_time: | tempo do início do percurso (millisecond) | integer 64
end_time: | tempo do fim do percurso (millisecond) | float 64
distance: | distância percorrida (m) | float 64
saved_emissions: | quantidade de CO$_2$ poupado (g)| float 64
route: | dados geoespaciais | string

In [7]:
for var in df.columns[1:]:
  print(var)
  print(pd.Series(pd.unique(df[var])))
  print()

session_id
0         5a6b9477-1457-4813-88cb-1a0bf88e2282
1         2a57ae36-e551-4073-b8f4-9ef7e1ede81f
2         24b6a823-03eb-4270-b255-8e7940e6bd2b
3         0b85ebb4-f6b3-45b3-9f99-b89aaede71a0
4         a6722d8d-7c8a-42db-9590-96ccf92cb34e
                          ...                 
108476    b546ff85-c774-4dd3-8e5f-778c7ed0213b
108477    d66e8f89-c1f4-4ead-b5d3-cdb2255c7c9a
108478    54644b93-648c-4b95-85ef-dd5c2c1e00c6
108479    4383e5e7-4fcf-419c-99d8-4adb1cee0431
108480    da6c04a6-40fd-4154-b7a4-667996acd842
Length: 108481, dtype: object

type
0    E-SCOOTER
1      SCOOTER
2         BIKE
dtype: object

start_time
0         1559070584000
1         1560191671000
2         1559738931000
3         1559519594000
4         1560097302000
              ...      
107171    1576657538000
107172    1576659615000
107173    1577783030000
107174    1578574271000
107175    1577600150000
Length: 107176, dtype: object

end_time
0         1559070951000
1         1560191872000
2         155

In [10]:
sum(df.start_time.isnull())

0

In [16]:
sum(df.end_time.isnull())

0

In [12]:
pd.__version__

'1.3.4'

In [15]:
pd.to_datetime(df['start_time'][0], unit='ms')

Timestamp('2019-05-28 19:09:44')

In [13]:
import time

df['start_time']= pd.to_datetime(df['start_time'], unit= 'ms')
df['end_time']= pd.to_datetime(df['end_time'], unit= 'ms')

ValueError: non convertible value NULL with the unit 'ms'

In [21]:
df.head()

,Unnamed: 0,vehicle_id,session_id,type,start_time,end_time,distance,saved_emissions,route
0,0,256,5a6b9477-1457-4813-88cb-1a0bf88e2282,E-SCOOTER,2019-05-28 19:09:44,2019-05-28 19:15:51,1472.0,135.04128,NaN
1,1,256,2a57ae36-e551-4073-b8f4-9ef7e1ede81f,E-SCOOTER,2019-06-10 18:34:31,2019-06-10 18:37:52,80.0,7.36720,01040000004A0000000101000000000000A0BD6E21C000...
2,2,256,24b6a823-03eb-4270-b255-8e7940e6bd2b,E-SCOOTER,2019-06-05 12:48:51,2019-06-05 12:58:48,2441.0,224.79169,0104000000490000000101000000000000800A6121C000...
3,3,256,0b85ebb4-f6b3-45b3-9f99-b89aaede71a0,E-SCOOTER,2019-06-02 23:53:14,2019-06-03 00:11:31,3867.0,354.75858,0104000000730000000101000000000000A00B6121C000...
4,4,256,a6722d8d-7c8a-42db-9590-96ccf92cb34e,E-SCOOTER,2019-06-09 16:21:42,2019-06-09 16:24:10,91.0,8.38019,01040000002A0000000101000000000000A0D27021C000...


In [13]:
time.strftime("%a, %d %b %Y %H:%M:%S +0000", time.localtime(df.start_time[0]))

'Tue, 28 May 2019 20:09:44 +0000'

In [11]:
time.strftime("%a, %d %b %Y %H:%M:%S +0000", time.localtime(df.end_time[0]))

'Tue, 28 May 2019 20:15:51 +0000'

In [17]:
from shapely import wkt

In [18]:
import geopandas as gpd

In [25]:
gpd.tools.geocode(df['route'][1])

,geometry,address
0,GEOMETRYCOLLECTION EMPTY,None


In [62]:
import matplotlib.pyplot as plt
from matplotlib import rcParams

rcParams.update({'figure.autolayout': True})
# set dimensions for printing
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 100)
pd.set_option('display.max_colwidth', None)#-1)